# Algorytm symulowanego wyżarzania (SA)

Grupa: Amelia Madej, Justyna Sarkowicz, Olga Sieradzan, Weronika Duda i Aleksandra Węgrzyn

## Biblioteki

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

## Pobór danych

In [ ]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaj ruchów

Poniżej znajduje się implementacja trzech rodzajów ruchów (generowania rozwiązań sąsiednich)

In [ ]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Długość ścieżki

In [ ]:
# Funkcja obliczająca całkowity koszt ścieżki
# Suma odległości pomiędzy kolejnymi miastami oraz powrót do miasta początkowego

def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Implementacja algorytmu

In [ ]:
def simulated_annealing(
    distance_matrix,
    initial_temperature=1000,
    cooling_rate=0.99,
    max_iterations=10000,
    min_temperature=0.00001,
    time_limit=None,
    move_type="two_opt"  
):
    start_time = time.time()
    n = len(distance_matrix)

    # Losowe wygenerowanie początkowej trasy
    initial_route = list(range(n))
    random.shuffle(initial_route)
    
    # Mapowanie typu ruchu na odpowiednią funkcję generatora sąsiadów
    move_generators = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }

    if move_type not in move_generators:
        raise ValueError(f"Nieznany typ ruchu: {move_type}")

    move_generator = move_generators[move_type]

    # Inicjalizacja bieżącego i najlepszego rozwiązania
    current_solution = initial_route
    current_cost = calculate_path_cost(distance_matrix, current_solution)
    best_solution = current_solution[:]
    best_cost = current_cost

    # Ustawienie początkowej temperatury
    temperature = initial_temperature

    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            print("Przekroczono limit czasu.")
            break
        
        # Sprawdzenie, czy temperatura osiągnęła wartość minimalną
        if temperature < min_temperature:
            print("Temperatura osiągnęła wartość minimalną.")
            break

        # Generowanie sąsiada na podstawie wybranego typu ruchu
        neighbors = list(move_generator(current_solution))
        neighbor = random.choice(neighbors)
        neighbor_cost = calculate_path_cost(distance_matrix, neighbor)

        
        # Różnica w kosztach między bieżącym a sąsiednim rozwiązaniem
        delta = neighbor_cost - current_cost

        if delta < 0:
            current_solution = neighbor
            current_cost = neighbor_cost
        else:
            # Akceptacja gorszego rozwiązania z prawdopodobieństwem zależnym od temperatury
            acceptance_probability = math.exp(-delta / temperature)
            if random.random() < acceptance_probability:
                current_solution = neighbor
                current_cost = neighbor_cost


        # Aktualizacja najlepszego rozwiązania, jeśli obecne jest lepsze
        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_cost = current_cost

        # Schładzanie temperatury
        temperature *= cooling_rate

        if iteration % 100 == 0:
            print(f"Iteracja {iteration}, najlepszy koszt: {best_cost}, temperatura: {temperature:.2f}")

        total_time = time.time() - start_time

    return best_solution, best_cost, total_time

## Generowanie rozwiązań

Poniżej znajduje się zestawienie wyników z uwzględnieniem wpływu na wyniki różnych wartości parametrów algorytmu

Za podstawowe dane przyjmujemy :

* initial_temperature = 10000

* cooling_rate = 0.99

* max_iterations = 10000

* min_temperature = 0.00001

* time_limit = `None`

* move_type = `two_opt`


In [ ]:
def run_multiple_tests(data, dataset_name, num_tests=10):
    results = []
    for _ in range(num_tests):
        best_path, best_cost, best_time = simulated_annealing(data)
        results.append({
            "WYNIK": best_cost,
            "ŚCIEŻKA": best_path,
            "CZAS": best_time,
            "DATASET": dataset_name
        })
    return results

In [ ]:
all_results = []
all_results.extend(run_multiple_tests(data1, "DATA1"))
all_results.extend(run_multiple_tests(data2, "DATA2"))
all_results.extend(run_multiple_tests(data3, "DATA3"))

basic = pd.DataFrame(data = all_results)

with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='w') as writer:
    basic.to_excel(writer, sheet_name = "Basic", index=False)

Badanie wpływu parametru *move_type*

In [ ]:
def test_move_type(data, dataset_name, num_repeats=10):
    results = []
    move_types = ["swap", "two_opt", "insertion"]
    for move_type in move_types:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, move_type=move_type)
            results.append({"PARAMETR": move_type, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_move_type_1 = test_move_type(data1, "DATA1")
df_move_type_2 = test_move_type(data2, "DATA2")
df_move_type_3 = test_move_type(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
neighborhood_type_results = pd.concat([df_move_type_1, df_move_type_2, df_move_type_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    neighborhood_type_results.to_excel(writer, sheet_name="move_type", index=False)

Badanie wpływu zmian wartości parametru *initial_temperature*

In [ ]:
def test_initial_temperature(data, dataset_name, num_repeats=10):
    results = []
    initial_temperatures = [1000, 3000, 5000, 10000, 30000, 50000, 100000]
    for initial_temperature in initial_temperatures:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, initial_temperature=initial_temperature)
            results.append({"PARAMETR": num_starts, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_initial_temperature_1 = test_initial_temperature(data1, "DATA1")
df_initial_temperature_2 = test_initial_temperature(data2, "DATA2")
df_initial_temperature_3 = test_initial_temperature(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_initial_temperature_1, df_initial_temperature_2, df_initial_temperature_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "initial_temperature", index=False)

Badanie wpływu zmian wartości parametru *cooling_rate*

In [ ]:
def test_cooling_rate(data, dataset_name, num_repeats=10):
    results = []
    for cooling_rate in np.arange(0.8, 0.99, 0.01):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, cooling_rate=cooling_rate)
            results.append({"PARAMETR": num_starts, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_cooling_rate_1 = test_cooling_rate(data1, "DATA1")
df_cooling_rate_2 = test_cooling_rate(data2, "DATA2")
df_cooling_rate_3 = test_cooling_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_cooling_rate_1, df_cooling_rate_2, df_cooling_rate_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "cooling_rate", index=False)

Badanie wpływu zmian wartości parametru *max_iterations*

In [ ]:
def test_max_iterations(data, dataset_name, num_repeats=10):
    results = []
    for max_iteration in range(50, 500, 50):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, max_iteration=max_iteration)
            results.append({"PARAMETR": num_starts, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_max_iterations_1 = test_max_iterations(data1, "DATA1")
df_max_iterations_2 = test_max_iterations(data2, "DATA2")
df_max_iterations_3 = test_max_iterations(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_max_iterations_1, df_max_iterations_2, df_max_iterations_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "max_iterations", index=False)

Badanie wpływu zmian wartości parametru *min_temperature*

In [ ]:
def test_min_temperature(data, dataset_name, num_repeats=10):
    results = []
    for min_temperature in np.logspace(-6, -2, num=10):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, min_temperature=min_temperature)
            results.append({"PARAMETR": num_starts, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_min_temperature_1 = test_min_temperature(data1, "DATA1")
df_min_temperature_2 = test_min_temperature(data2, "DATA2")
df_min_temperature_3 = test_min_temperature(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_min_temperature_1, df_min_temperature_2, df_min_temperature_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "min_temperature", index=False)